In [2]:
import ccxt
import pandas as pd

# 定義交易對
symbol = 'PENGU/USDT'  # 假設 PENGU 是以 USDT 計價的
listing_time = '2023-01-01T00:00:00Z'  # 假設上幣日期
since = None

# 獲取幣種的上幣時間
def get_since(exchange):
    return exchange.parse8601(listing_time)

# 獲取歷史交易數據
def fetch_ohlcv(exchange, symbol):
    since = get_since(exchange)
    ohlcv = exchange.fetch_ohlcv(symbol, timeframe='1d', since=since)
    df = pd.DataFrame(ohlcv, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
    df['date'] = pd.to_datetime(df['timestamp'], unit='ms')
    df.drop(columns=['timestamp'], inplace=True)
    return df

# 初始化交易所
exchanges = {
    'binance': ccxt.binance(),
    #'okx': ccxt.okx(),
    # 'bybit': ccxt.bybit(),
}

# 獲取數據並保存為 CSV
for exchange_name, exchange in exchanges.items():
    try:
        df = fetch_ohlcv(exchange, symbol)
        csv_filename = f'{exchange_name}_pengu_ohlcv_data.csv'
        df.to_csv(csv_filename, index=False)
        print(f"Data saved to {csv_filename}")
    except Exception as e:
        print(f"Error fetching data from {exchange_name}: {e}")

Data saved to binance_pengu_ohlcv_data.csv


In [3]:
import requests
import pandas as pd

# 定義 OKX API URL
base_url = 'https://www.okx.com/api/v5'

# 定義交易對和幣種上幣時間
symbol = 'PENGU-USDT'  # 假設 PENGU 是以 USDT 計價的
listing_time = '2023-01-01T00:00:00Z'  # 假設上幣日期

# 將上幣時間轉換為毫秒時間戳
since = int(pd.to_datetime(listing_time).timestamp() * 1000)

# 獲取歷史 OHLCV 數據
def fetch_ohlcv(symbol, since):
    url = f'{base_url}/market/candles?instId={symbol}&bar=1D&start={since}'
    response = requests.get(url)

    # 檢查請求是否成功
    if response.status_code == 200:
        data = response.json()
        if data['code'] == '0':
            return data['data']
        else:
            print(f"Error: {data['msg']}")
            return None
    else:
        print(f"HTTP Error: {response.status_code}")
        return None

# 獲取數據並保存為 CSV
ohlcv_data = fetch_ohlcv(symbol, since)
if ohlcv_data:
    # 將數據轉換為 DataFrame
    df = pd.DataFrame(ohlcv_data, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_volume', 'count'])
    
    # 將 timestamp 轉換為可讀的日期時間格式
    df['date'] = pd.to_datetime(df['timestamp'], unit='ms')

    # 刪除不需要的列
    df.drop(columns=['timestamp', 'close_time', 'quote_volume', 'count'], inplace=True)

    # 顯示 DataFrame
    print(df)

    # 將 DataFrame 保存為 CSV 文件
    df.to_csv('okx_pengu_ohlcv_data.csv', index=False)
    print("Data saved to okx_pengu_ohlcv_data.csv")

       open      high       low     close      volume                date
0  0.034532    0.0395  0.030616  0.032273  2332995734 2024-12-18 16:00:00
1  0.036388   0.03973  0.026222  0.034563  2811583370 2024-12-17 16:00:00
2     0.005  0.056334     0.005  0.036417   804762961 2024-12-16 16:00:00
Data saved to okx_pengu_ohlcv_data.csv


/var/folders/ks/h0cwv1cn5t92_dn23lyd5_kw0000gn/T/ipykernel_2528/8600489.py:38: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  df['date'] = pd.to_datetime(df['timestamp'], unit='ms')


In [5]:
import ccxt
import csv
from pprint import pprint

# 创建 Bybit 实例，替换为您的 API 密钥和秘密
bybit = ccxt.bybit()

# 加载市场信息
markets = bybit.load_markets()

# 获取 OHLCV 数据
symbol = 'PENGU/USDT'  # 确保符号格式正确
try:
    ohlcv = bybit.fetch_ohlcv(symbol, timeframe='1d', limit=200)
    print(ohlcv)

    # 保存为 CSV 文件
    with open('bybit_pengu_ohlcv_data.csv', mode='w', newline='') as file:
        writer = csv.writer(file)
        # 写入表头
        writer.writerow(['timestamp', 'open', 'high', 'low', 'close', 'volume'])
        # 写入数据
        writer.writerows(ohlcv)

    print("OHLCV 数据已成功保存为 ohlcv_data.csv")
except Exception as e:
    print(f"Error fetching OHLCV data: {e}")


[[1734393600000, 0.0034, 0.05851, 0.0034, 0.02875, 1236362289.0], [1734480000000, 0.02875, 0.03949, 0.0265, 0.03559, 1779231684.0], [1734566400000, 0.03559, 0.03694, 0.03061, 0.03223, 511658641.0]]
OHLCV 数据已成功保存为 ohlcv_data.csv
